# bcftools mpileup

Called SNPs by comparing all files to the reference genome

In [ ]:
#!/bin/bash

module load samtools
module load bcftools

SAMPLELIST="../01-trim_qc/x${SLURM_ARRAY_TASK_ID}.fl"

for SAMPLE in `cat $SAMPLELIST`; do


bcftools mpileup -a AD,ADF,DP,ADR,SP -Ou -f ../ref_genome/ref_genome.fa ./sorted/${SAMPLE}_sorted_minq20.bam | bcftools call --threads 8 -mv -Oz -f GQ -o ./GQmpileup/${SAMPLE}.vcf

done

# vcftools filter

Filtered the files all together with VCFtools

In [ ]:
#!/bin/bash

module load vcftools

vcftools --vcf filename.vcf --minDP 5 --maxDP 200 --max-missing 0.9 --maf 0.05 --minQ 20 --remove-indels --remove-filtered-all --recode --recode-INFO-all --out filtered_filename


# bcftools reheader

Reheader samples in .vcf file to correct sample names

In [ ]:
#!/bin/bash

module load bcftools

bcftools reheader -s filelist.txt filtered_filename.vcf -o reheadered_filtered_filename.vcf


# bcftools sort and index

Sort and index the population .vcf file

In [ ]:
#!/bin/bash

module load samtools
module load bcftools


bcftools sort reheadered_filtered_filename.vcf -Oz z -o reheadered_filtered_filename_sorted.vcf.gz
bcftools index -t reheadered_filtered_filename_sorted.vcf.gz

# Beagle-haplotype phasing

Phase the whole population file

In [ ]:
#!/bin/bash

module load beagle/5.4
java -Xmx100g -jar ${EBROOTBEAGLE}/beagle.22Jul22.46e.jar
java -Xmx100g -jar ${EBROOTBEAGLE}/beagle.22Jul22.46e.jar gt=reheadered_filtered_filename_sorted.vcf.gz window=300 excludemarkers=./markers.txt impute=false out=phased_filename

# Split population file into individual .vcf files

In [ ]:
#!/bin/bash

module load bcftools

for file in phased_filename.vcf.gz; do
  for sample in `bcftools query -l $file`; do
    bcftools view -Oz -s $sample -o ${file/.vcf*/.$sample.vcf.gz} $file
  done
done


# bcftools merge

Merge individual .vcf files into two separate populations (non-survivors vs. survivors & immune responders) for comparison

In [ ]:
#!/bin/bash


module load bcftools

bcftools merge --file-list ./filelist.txt -Oz -o merged_populationfile.vcf.gz


# samtools faidx and bcftools consensus

Create consensus sequences based on the reference genome, and output a mega file containing both haplotypes for further analysis. Code adapted from Magid et al. (2022).

In [ ]:
#!/bin/bash


module load samtools
module load bcftools

counter=1 #counter adds unique number labels to each individual for MEGA format


echo "#MEGA" > tlr_haplotypes.meg
#for loop to output haplotypes for each individual bam file within the final vcf for one TLR region
for file in *.bam #loop through the population bam files
do
base=$(basename $file.bam)
echo "#"${counter}"-1" >> tlr_haplotypes.meg
#outputs sequence for phased haplotype 1 for one individual
samtools faidx reference.genome.fa chr:1-2 | bcftools consensus phased_filename.vcf.gz -s $base -M N -H 1pIu >> tlr_haplotypes.meg
echo "#"${counter}"-2" >> tlr_haplotypes.meg
#outputs sequence for phased haplotype 2 for one individual
samtools faidx referencegenome.fa chr:1-2 | bcftools consensus phased_filename.vcf.gz -s $base -M N -H 2pIu >> tlr_haplotypes.meg
counter=$(($counter+1))
done

.meg files with haplotype sequences were imported into DNAsp for analysis of haplotype diversity, nucleotide diversity, and Tajima's D, using the "DNA Polymorphism" and "Tajima's Test" commands.